<a href="https://colab.research.google.com/github/Kazuto-Takahashi/Research/blob/main/SCNN_Con_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import snntorch
import snntorch.functional as SF

**Data**

In [ ]:
import torchvision
import torchvision.transforms.v2 as transforms
from torchvision.datasets import CIFAR10

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#CIFAR10 := 10classes, 3x32x32, 50000train, 10000test, 6000image/class
train = CIFAR10(root='./data', train=True, download=True, transform=transform)
test = CIFAR10(root='./data', train=False, download=True, transform=transform)

#loader := [c, h, w] = [3, 32, 32]
train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


**Model**

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 8, 3) # [3, 32, 32] -> [8, 30, 30]
    self.pooling1 = nn.Conv2d(8, 8, 4, 4) #maxpooling, (in, out, kernel, stride), [8, 30, 30] -> [8, 15, 15]
    self.conv2 = nn.Conv2d(8, 32, 4) # [8, 15, 15] -> [32, 12, 12]
    self.pooling2 = nn.Conv2d(32, 32, 2, 2) #maxpooling, [32, 12, 12] -> [32, 6, 6]
    self.conv3 = nn.Conv2d(32, 128, 6) # image = [1, 1]となるようにカーネルサイズ調整
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(128, 10)

  def forward(self, x): # [b, c, h, w] -> [b, class]
    x = torch.abs(x)
    x = F.relu(self.conv1(x))
    x = self.pooling1(x)
    x = F.relu(self.conv2(x))
    x = self.pooling2(x)
    x = F.conv3(x)
    print(f'conv3あと, {x.shape}')
    x = self.flatten(x)
    x = self.fc(x)
    return

net = Net()

**Other Settings**

In [ ]:
optimizer = optim.Adam(net.parameters(), betas=(0.9, 0.999))
loss_fn = F.CrossEntropyLoss()
acc_fn = F.accuracy_score
device = 'cpu'

**Training**

In [ ]:
epochs = 4

for epoch in range(epochs):
  loss_train = 0
  net.train()
  for data, target in train_loader:
    data = data.to(device)
    target = target.to(device)

    net.zero_grad()
    output = net(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

    loss_train += loss.item()
    y = torch.argmax(output, dim=1)
    acc =

  with torch.no_grad():
    net.eval()
    loss_test = 0
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)

      output = net(data)
      loss = loss_fn(output, target)
      loss_test += loss.item()
    print(f'epoch{epoch}, test')